## Import important Libraries

In [1]:
import pandas as pd
import pymysql as sql

## Youtube_API.csv contains data pulled from the YouTube API.
### We will us this dataframe to push data into a table on AWS cloud Database

In [2]:
df = pd.read_csv('Youtube_API.csv', index_col=0)
df

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,F2yswKCPGoI,Jimmy Kimmel Has Had Enough of Scumbags Attack...,2021-12-01,1197109,35000,2309,4126
1,6ilZeMFyNE8,LN+ EN VIVO las 24 horas,2021-11-01,9714535,25110,3463,0


### To connect to the cloud database we need a username, password, host, database_name and port
#### We define a function to connect to the cloud database and raise an error message if any...

In [3]:
## This will try and let us know if connection was possible or there was any error..

def connect_to_db(username, password, host_name, dbname, port):
    try:
        conn = sql.connect(user=username, password=password, host=host_name, database=dbname, port=port)

    except sql.OperationalError as e:
        raise e

    else:
        print ('Connected!!!')
    
    return conn

In [4]:
## All this data is confidential and must not be shared...

host_name = "**********************"
dbname = '***********'
port = 3306
username = '*******'
password = '*********'
conn = None

conn = connect_to_db(username, password, host_name, dbname, port)

Connected!!!


In [5]:
## cursor is used to run sql queries in python environment
curr = conn.cursor()

In [6]:
# We need to create a table in our AWS cloud database where we can store our data.

def create_table(curr):
    create_table_command = (""" CREATE TABLE IF NOT EXISTS videos(
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT (CURRENT_DATE),
                    view_count INT NOT NULL,
                    like_count INT NOT NULL,
                    dislike_count INT NOT NULL,
                    comment_count INT NOT NULL )""")
    curr.execute(create_table_command)

In [7]:
create_table(curr)

In [8]:
curr.execute("SHOW COLUMNS FROM videos;")

7

## In order to push data to cloud and make sure there are no duplicates we need to check if any video already exists in the database and if it exists we update the table and if not we insert the new row in the table.

In [9]:
# Check if the video exists

def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id FROM videos WHERE video_id = %s;""")
    curr.execute(query, (video_id)) #The values for placeholders (%s) will always be passed as tuples
    
    return curr.fetchone() is not None

In [10]:
# Update row if the video exists

def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
            view_count = %s,
            like_count = %s,
            dislike_count = %s,
            comment_count = %s
            WHERE video_id = %s;""")

    vars_to_update = (video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query,vars_to_update)

In [11]:
# Function to update or insert new values. We combine all the above functions.
## We also create a temporary table to store new rows which we will use to append to the cloud.

def update_db(curr,df):
    # temporary dataframe to store new rows
    tmp_df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count',
                              'dislike_count','comment_count'])
   
    for i, row in df.iterrows(): # df.iterrows() to apply for loop on pandas df
        
        if check_if_video_exists(curr, row['video_id']): # If the video exists we will update
            update_row(curr,row['video_id'], row['video_title'],row['view_count'],row['like_count'],
                       row['dislike_count'],row['comment_count'])
        else: # If the video doesn't exist we append those to a new dataframe
            tmp_df = tmp_df.append(row)
    
    return tmp_df

In [12]:
updated_df = update_db(curr,df)

### After getting the new rows and updating any old rows we use a function to update our table in cloud.

In [13]:
def insert_into_table(curr, video_title, view_count, upload_date, like_count,
                      dislike_count, comment_count, video_id):
    
    query = ("""INSERT INTO videos
            (video_id, video_title,upload_date,view_count,like_count,dislike_count,comment_count)
            VALUES (%s,%s,%s,%s,%s,%s,%s)""")

    row_to_insert=(video_title, view_count, upload_date, like_count, dislike_count, comment_count, video_id)
    curr.execute(query,row_to_insert)

In [14]:
def df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['view_count'], row['upload_date'], 
                          row['like_count'], row['dislike_count'], row['comment_count'] )

In [15]:
df_to_db(curr, updated_df)

In [16]:
conn.commit()